# Dimensionality Reduction - Autoencoder
> [Alik604](https://github.com/alik604/youtube)

In [1]:
%%capture

# !pip install mlxtend catboost scikit-plot xgboost lightgbm

import xgboost  # xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
import lightgbm # lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
import catboost # catboost.ai/docs/concepts/python-quickstart.html

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import * 
from sklearn.preprocessing import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss

from mlxtend.classifier import EnsembleVoteClassifier
import copy

import matplotlib.pyplot as plt
import scikitplot as skplt

from scipy.io import loadmat

import warnings
warnings.simplefilter('ignore')

In [2]:
mat = loadmat('arrhythmia.mat')
# SRC: http://odds.cs.stonybrook.edu/arrhythmia-dataset/
# Description: X = Multi-dimensional point data, y = labels (1 = outliers, 0 = inliers)

X = mat['X']
y = mat['y']

print(X.shape)
print(y.shape)
y = y.ravel()
print(y.shape)

(452, 274)
(452, 1)
(452,)


In [3]:
portion_of_outliers = np.sum(y)/len(y) # (1 = outliers, 0 = inliers)
print("portion_of_outliers", portion_of_outliers)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

portion_of_outliers 0.14601769911504425


In [4]:
wanted_explained_variance_ratio = 0.99
steps_down = 20
wanted_n_components = X_train.shape[1]
first_time = True

for i in range(X_train.shape[1]-1, 1, -steps_down):
  total_var_ratio = round(np.sum(PCA(n_components=i).fit(X_train).explained_variance_ratio_), 5)
  print('i =', i, 'with a variance ratio of', total_var_ratio)
  if total_var_ratio < wanted_explained_variance_ratio and first_time:
    wanted_n_components = i + steps_down
    first_time = False
#     break

print("We should set n_components to: ", wanted_n_components)

i = 273 with a variance ratio of 1.0
i = 253 with a variance ratio of 1.0
i = 233 with a variance ratio of 1.0
i = 213 with a variance ratio of 1.0
i = 193 with a variance ratio of 1.0
i = 173 with a variance ratio of 1.0
i = 153 with a variance ratio of 0.99999
i = 133 with a variance ratio of 0.99996
i = 113 with a variance ratio of 0.99981
i = 93 with a variance ratio of 0.99906
i = 73 with a variance ratio of 0.99562
i = 53 with a variance ratio of 0.9829
i = 33 with a variance ratio of 0.93978
i = 13 with a variance ratio of 0.77919
We should set n_components to:  73


In [7]:
def benchmark(X_train, X_test, y_train = y_train, y_test = y_test, verbose = False, scale = False, run_PCA = None, title = ''):    
    print("\n=======================================")
    print(title)
    print("---------------------------------------")

    if scale:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
    if run_PCA:
        pca = PCA(n_components=run_PCA)
        _ = pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test = pca.transform(X_test)

    DTC = DecisionTreeClassifier() 
    RFC = RandomForestClassifier(n_estimators=250, n_jobs=-1)
    ETC = ExtraTreesClassifier(n_estimators=250, n_jobs=-1)
    XGB = xgboost.XGBClassifier(n_estimators=250, n_jobs=-1)
    GBM = lightgbm.LGBMClassifier(n_estimators=200, n_jobs=-1)
#     RFC = RandomForestClassifier(n_estimators=250, random_state=42, n_jobs=-1)
#     ETC = ExtraTreesClassifier(n_estimators=250, random_state=42, n_jobs=-1)
#     XGB = xgboost.XGBClassifier(n_estimators=250, random_state=42, n_jobs=-1)
#     GBM = lightgbm.LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)

    list_of_CLFs_names = []
    list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
    ranking = []

    for clf in list_of_CLFs: 
        _ = clf.fit(X_train,y_train)
        pred = clf.score(X_test,y_test)
        name = str(type(clf)).split(".")[-1][:-2]
        if verbose:
            print("Acc: %0.5f for the %s" % (pred, name))
        
        ranking.append(pred)
        list_of_CLFs_names.append(name)

    # drop any subpar classifier 
    best = np.max(ranking)
    avg = np.sum(ranking)/len(ranking)
    variance = best - avg 
    to_remove = ranking - avg + variance
    to_remove_alt = ranking - best + variance
    # print(list_of_CLFs_names)
    # print(to_remove)      
    # print(to_remove_alt)

    ranking = np.array(ranking)[to_remove > 0]
    list_of_CLFs = np.array(list_of_CLFs)[to_remove > 0]

    eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, refit=False, voting='soft')
    eclf.fit(X_train, y_train)
    pred = eclf.predict(X_test)
    probas = eclf.predict_proba(X_test)
    acc = eclf.score(X_test, y_test)
    print("---------------------------------------")
    print("Acc: %0.5f for the %s" % (acc, str(type(eclf)).split(".")[-1][:-2]))
#     print("F1 score: \t\t", round(f1_score(y_test, pred, average='micro'), 3))                # 2 * (precision * recall) / (precision + recall)
    print("Log loss (categorical cross entropy): \t", round(log_loss(y_test, probas), 3)) # -log P(yt|yp) = -(yt log(yp) + (1 - yt) log(1 - yp))

    if verbose:
        skplt.metrics.plot_roc(y_true=y_test, y_probas=probas)
        plt.show()

In [30]:
benchmark(X_train, X_test, y_train, y_test, scale = False, run_PCA = False, title= "No Scaling, no PCA, Dim:" + str(X_train.shape[1]) )
benchmark(X_train, X_test, y_train, y_test, scale = True , run_PCA = False, title= "Scaling, no PCA")

benchmark(X_train, X_test, y_train, y_test, scale = False, run_PCA = wanted_n_components, title= "No scaling, PCA:" +str(wanted_n_components))
benchmark(X_train, X_test, y_train, y_test, scale = True, run_PCA = wanted_n_components, title= "Scaling, PCA:" +str(wanted_n_components))


No Scaling, no PCA, Dim:274
---------------------------------------
---------------------------------------
Acc: 0.94505 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.171

Scaling, no PCA
---------------------------------------
---------------------------------------
Acc: 0.95604 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.18

No scaling, PCA:73
---------------------------------------
---------------------------------------
Acc: 0.90110 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.27

Scaling, PCA:73
---------------------------------------
---------------------------------------
Acc: 0.90110 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.276


In [31]:
benchmark(X_train, X_test, y_train, y_test, scale = False, run_PCA = 30, title= "No scaling, PCA:" +str(30))
benchmark(X_train, X_test, y_train, y_test, scale = False, run_PCA = 2, title= "No scaling, PCA:" +str(2))
benchmark(X_train, X_test, y_train, y_test, scale = True, run_PCA = 2, title= "Scaling, PCA:" +str(2))


No scaling, PCA:30
---------------------------------------
---------------------------------------
Acc: 0.91209 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.269

No scaling, PCA:2
---------------------------------------
---------------------------------------
Acc: 0.85714 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.475

Scaling, PCA:2
---------------------------------------
---------------------------------------
Acc: 0.85714 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.478


### Scale our data

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Let's try a autoenoder to 'beat' the proformance of PCA

In [12]:
import tensorflow
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
model = Sequential()

model.add(Dense(512,  activation='elu', input_shape=(X_train.shape[1],)))
model.add(Dense(128,  activation='elu'))

model.add(Dense(2,    activation='linear', name="bottleneck")) # elu activation yeilds [-1, -1, NUMB]

# model.add(Dropout(0.4))
model.add(Dense(128,  activation='elu'))
model.add(Dense(512,  activation='elu'))
model.add(Dense(X_train.shape[1],  activation='elu'))
model.compile(loss='mean_squared_error', optimizer = Adam(0.0005))


ES = EarlyStopping(patience=7, restore_best_weights=True)
history = model.fit(X_train, X_train, batch_size=4, epochs=250, verbose=1, validation_split = 0.1, callbacks =[ES])
print(f'MSE: {np.sum(((X_train-model.predict(X_train))**2).mean(axis=1)):.2f}')

encoder = Model(model.input, model.get_layer('bottleneck').output)
embedding = encoder.predict(X_train)  # bottleneck representation


#Acc: 0.85714 for the EnsembleVoteClassifier | PCA(2) | Log loss: 0.487
#Acc: 0.89011 for the EnsembleVoteClassifier | AE(2) | Log loss: 0.498

Train on 324 samples, validate on 37 samples
Epoch 1/250
324/324 [==============================] - 0s 2ms/sample - loss: 0.8905 - val_loss: 0.8981
Epoch 2/250
324/324 [==============================] - 0s 557us/sample - loss: 0.8553 - val_loss: 0.8891
Epoch 3/250
324/324 [==============================] - 0s 559us/sample - loss: 0.8151 - val_loss: 0.8827
Epoch 4/250
324/324 [==============================] - 0s 574us/sample - loss: 0.7982 - val_loss: 0.8886
Epoch 5/250
324/324 [==============================] - 0s 515us/sample - loss: 0.7958 - val_loss: 0.8880
Epoch 6/250
324/324 [==============================] - 0s 485us/sample - loss: 0.7798 - val_loss: 0.8810
Epoch 7/250
324/324 [==============================] - 0s 515us/sample - loss: 0.7737 - val_loss: 0.8870
Epoch 8/250
324/324 [==============================] - 0s 556us/sample - loss: 0.7726 - val_loss: 0.8999
Epoch 9/250
324/324 [==============================] - 0s 549us/sample - loss: 0.7650 - val_loss: 0.8800
Epoch 10/250

In [22]:
X_train_dimRect_via_AE = encoder.predict(X_train)
X_test_dimRect_via_AE = encoder.predict(X_test)
print(X_train_dimRect_via_AE.shape)

print(X_train_dimRect_via_AE[1])

(361, 2)
[-1.7372092  1.3490907]


In [32]:
benchmark(X_train, X_test, y_train, y_test, scale = False, run_PCA = 2, title= "PCA: " + str(2)) # PCA(2): 89%
benchmark(X_train_dimRect_via_AE, X_test_dimRect_via_AE, y_train, y_test, scale = False, run_PCA = False, title= "Autoencoder's embedding dim: " + str(X_train_dimRect_via_AE.shape[1]))


PCA: 2
---------------------------------------
---------------------------------------
Acc: 0.85714 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.5

Autoencoder's embedding dim: 2
---------------------------------------
---------------------------------------
Acc: 0.87912 for the EnsembleVoteClassifier
Log loss (categorical cross entropy): 	 0.381


In [24]:
# 92% with PCA - 0.99
# 94% without PCA NOR Standard Scaler 
# 94% with only Standard Scaler 

## best out of 5 tested
# Acc: 0.82418 for the LGBMClassifier on 2D VIA AE - few epochs 
# Acc: 0.79121 for the LGBMClassifier on 2D VIA AE - many epochs
# Acc: 0.81319 for the LGBMClassifier on 2D VIA AE - many epochs & Dropout(0.4)
# Acc: 0.85714 for the LGBMClassifier on 2D VIA AE - many epochs & Dropout(0.4) & Smaller batch_size (128 -> 16)
# Acc: 0.86813 for the LGBMClassifier on 2D VIA AE - many epochs & Smaller batch_size (128 -> 16);  Acc: 0.90110 for the DecisionTreeClassifier
# Acc: 0.89011 for the ExtraTreesClassifier        - via chance??  
# Acc: 0.93407 for the XGBClassifier               - many epochs & Dropout(0.4) & Smaller batch_size 

# Acc: 0.83516 for the LGBMClassifier on 2D via PCA


# output of PCA(2)
# Acc: 0.75824 for the DecisionTreeClassifier
# Acc: 0.86813 for the RandomForestClassifier
# Acc: 0.83516 for the ExtraTreesClassifier
# Acc: 0.86813 for the XGBClassifier
# Acc: 0.83516 for the LGBMClassifier

In [28]:
# # much faster 
# mses = ((X_train-model.predict(X_train))**2).mean(axis=1)
# np.sum(mses)

# count = 0
# pred = model.predict(X_train)
# for i in range(361):
#     for j in range(X_train.shape[1]):
#         count += (X_train[i][j] - pred[i][j])**2
# count /= X_train.shape[0]

268.450175439129